<a href="https://colab.research.google.com/github/yunmokoo/SPY-Forecast_MARS_LSTM/blob/main/SPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install requests > make_outputrerquest.txt 2>&1
!pip install yfinance > make_outputyfinance.txt 2>&1
!pip install transformers > make_outputtransformers.txt 2>&1
!pip install xformers > make_outputxformers.txt 2>&1


In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz > wget_output.txt 2>&1
!tar -xzvf ta-lib-0.4.0-src.tar.gz > tar_output.txt 2>&1
%cd ta-lib
!./configure --prefix=/usr > config_output.txt 2>&1
!make > make_outputtalib.txt 2>&1
!make install > makeInstall_output.txt 2>&1
!pip install TA-Lib > pipInstall_output.txt 2>&1

/content/ta-lib


In [37]:
import requests
import json
import yfinance as yf
#import talib

import tensorflow as tf
import numpy as np
import pandas as pd
import torch
import matplotlib as plt

from transformers import *
from transformers import GenerationMixin
from transformers import TFGenerationMixin
from transformers import FlaxGenerationMixin

from sklearn.preprocessing import MinMaxScaler

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

From TA-lib...

1. MA                  All Moving Average
2. BBANDS              Bollinger Bands
3. MACD                Moving Average Convergence/Divergence
4. SMA                 Simple Moving Average
5. EMA                 Exponential Moving Average

In [38]:

data_SPY = yf.download("SPY", start = "2001-01-01", end = "2023-01-01")


print(data_SPY)

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2001-01-02  132.000000  132.156250  127.562500  128.812500   84.662979   
2001-01-03  128.312500  136.000000  127.656250  135.000000   88.729759   
2001-01-04  134.937500  135.468750  133.000000  133.546875   87.774689   
2001-01-05  133.468750  133.625000  129.187500  129.187500   84.909462   
2001-01-08  129.875000  130.187500  127.687500  130.187500   85.566704   
...                ...         ...         ...         ...         ...   
2022-12-23  379.649994  383.059998  378.029999  382.910004  380.042480   
2022-12-27  382.790009  383.149994  379.649994  381.399994  378.543793   
2022-12-28  381.329987  383.390015  376.420013  376.660004  373.839264   
2022-12-29  379.630005  384.350006  379.079987  383.440002  380.568512   
2022-12-30  380.640015  382.579987  378.429

In [39]:
df = pd.DataFrame(data_SPY)
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2001-01-02,132.000000,132.156250,127.562500,128.812500,84.662979,8737500
2001-01-03,128.312500,136.000000,127.656250,135.000000,88.729759,19431600
2001-01-04,134.937500,135.468750,133.000000,133.546875,87.774689,9219000
2001-01-05,133.468750,133.625000,129.187500,129.187500,84.909462,12911400
2001-01-08,129.875000,130.187500,127.687500,130.187500,85.566704,6625300
...,...,...,...,...,...,...
2022-12-23,379.649994,383.059998,378.029999,382.910004,380.042480,59857300
2022-12-27,382.790009,383.149994,379.649994,381.399994,378.543793,51638200
2022-12-28,381.329987,383.390015,376.420013,376.660004,373.839264,70911500


# Transformer


In [40]:
scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_normalized

,Open,High,Low,Close,Adj Close,Volume
0,0.155737,0.151608,0.147845,0.148199,0.079748,0.006263
1,0.146771,0.160983,0.148074,0.163306,0.089547,0.018587
2,0.162880,0.159688,0.161140,0.159758,0.087246,0.006818
3,0.159308,0.155190,0.151818,0.149115,0.080342,0.011073
4,0.150570,0.146806,0.148150,0.151556,0.081925,0.003829
...,...,...,...,...,...,...
5530,0.757896,0.763598,0.760294,0.768555,0.791514,0.065175
5531,0.765531,0.763818,0.764256,0.764868,0.787902,0.055703
5532,0.761981,0.764403,0.756358,0.753296,0.776566,0.077915
5533,0.757848,0.766745,0.762862,0.769849,0.792781,0.073373


In [61]:
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim, output_dim, num_heads, hidden_dim, num_layers):
        super(TimeSeriesTransformer, self).__init__()

        # Input and output dimensions
        self.input_dim = input_dim
        self.output_dim = output_dim

        # Transformer layers
        self.transformer_layers = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads)
        self.transformer = nn.TransformerEncoder(self.transformer_layers, num_layers=num_layers)

        # Output layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.transformer(x)
        x = self.fc(x)
        return x

In [62]:
input_dim = 6  # Replace with the number of input features in your time series data
output_dim = 1  # Replace with the number of output classes or units (for regression)
num_heads = 8  # Choose the number of attention heads (e.g., 8 or 16)
hidden_dim = 128  # Choose the size of the hidden dimension (e.g., 128 or 256)
num_layers = 4  # Choose the number of Transformer layers (e.g., 4 or 6)

model = TimeSeriesTransformer(input_dim=input_dim, output_dim=output_dim, num_heads=num_heads,
                              hidden_dim=128, num_layers=num_layers)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adjust the learning rate as needed


criterion = nn.MSELoss()  # Replace with the appropriate loss function for your task (e.g., nn.MSELoss() for regression)


In [63]:
# Define the train-validation split ratio based on time (e.g., 80% training, 20% validation)
train_val_split_ratio = 0.8

# Determine the index that corresponds to the split point
split_index = int(train_val_split_ratio * len(df_normalized))

# Split the data into training and validation sets based on time
train_indices = df_normalized.index[:split_index]
val_indices = df_normalized.index[split_index:]

In [64]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx].values
        return torch.tensor(sample, dtype=torch.float32)

train_dataset = TimeSeriesDataset(df_normalized.loc[train_indices])
val_dataset = TimeSeriesDataset(df_normalized.loc[val_indices])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [74]:
num_epochs = 100  # Set the number of epochs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        inputs = batch['data']  # Extract the input data from the batch dictionary
        targets = batch['target']  # Extract the target values from the batch dictionary

        # Print the shape of inputs and targets
        print("Inputs shape:", inputs.shape)
        print("Targets shape:", targets.shape)

        inputs = inputs.squeeze(1).to(device)  # Squeeze extra dimensions and move to device

        optimizer.zero_grad()
        outputs = model(inputs)

        targets = targets.view(-1, 1).to(device)  # Reshape to match the shape of 'outputs'

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")


IndexError: ignored